# En el presente archivo es la conclusion del uso del sistema de recomendacion con la API de Flask

## Importaciones de las librerias necesarias para la API

In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

## Creacion de la API de flask con un endpoint metodo POST `/predict` en la cual se leera el `level`, `score` y `time` para pasarlo al sistema de recomendacion y obtener el resultado

In [ ]:
app = Flask(__name__)

models = {}
scalers = {}

for level in range(1, 7):
    models[level] = joblib.load(f'svm_model_grade_{level}.pkl')
    scalers[level] = joblib.load(f'scaler_grade_{level}.pkl')

@app.route('/predict', methods=['POST']) 
def predict():
    data = request.json
    level = data['level']
    score = data['score']
    time = data['time']

    normalized_score = score / average_scores_by_level[level]
    normalized_time = time / average_time_by_level[level]

    features = np.array([[normalized_score, normalized_time]])
    features = scalers[level].transform(features)

    model = models[level]
    prediction = model.predict(features)

    if prediction[0] == 'NO APRENDIO':
        recommendation = "Disminuye un nivel para consolidar conocimientos"
    else:
        # A nivel del videojuego se avanzara al siguiente nivel de manera automatica una vez leido el mensaje de respuesta
        recommendation = "Muy bien, avanza al siguiente nivel para seguir aprendiendo"

    return jsonify({
        'prediction': prediction[0],
        'recommendation': recommendation
    })

if __name__ == '__main__':
    app.run(debug=True)